In [12]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [13]:
df = pd.read_excel("data.xlsx", index_col="exp")
_X = df.iloc[:, :-3].values
_Y = df.iloc[:, -3:].values
print(_X.shape)
print(_Y.shape)

(100, 47)
(100, 3)


In [14]:
_X_train, _X_test, _Y_train, _Y_test = train_test_split(
    _X, _Y, test_size=0.3, random_state=0
)
print(_X_train.shape)
print(_X_test.shape)
print(_Y_train.shape)
print(_Y_test.shape)

(70, 47)
(30, 47)
(70, 3)
(30, 3)


In [15]:
scX = StandardScaler()
X_train = scX.fit_transform(_X_train)
X_test = scX.transform(_X_test)

scY = StandardScaler()
Y_train = scY.fit_transform(_Y_train)
Y_test = scY.transform(_Y_test)

In [16]:
X_train_pt = torch.from_numpy(X_train).float()
X_test_pt = torch.from_numpy(X_test).float()
Y_train_pt = torch.from_numpy(Y_train).float()
Y_test_pt = torch.from_numpy(Y_test).float()

In [17]:
print(type(X_train))
print(type(X_train_pt))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [18]:
num_features = _X.shape[1]
num_outputs = _Y.shape[1]

class MyModel(nn.Module):
    def __init__(self, num_features, num_outputs):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 24)
        self.fc2 = nn.Linear(24, 12)
        self.fc3 = nn.Linear(12, 6)
        self.fc4 = nn.Linear(6, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
model = MyModel(num_features=num_features, num_outputs=num_outputs)

In [ ]:
n_epochs = 100

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

loss_arr = []
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    Y_pred = model(X_train_pt)
    loss = loss_fn(Y_pred, Y_train_pt)
    loss.backward()
    optimizer.step()
    
    epoch_train_loss = loss.item()
    loss_arr.append(epoch_train_loss)

with torch.no_grad():
    test_pred = model(X_test_pt)
    final_loss = loss_fn(test_pred, Y_test_pt)

# Visualization
ax = sns.lineplot(loss_arr)  # plot loss over epochs using seaborn
ax.set_xlabel("Epoch")
ax.set_ylabel("MSE")
ax.set_title(f"Final Loss = {final_loss}")